In [1]:
# import used libraries 
import pandas as pd 
import numpy as np
import plotly.express as px 

import sys
from pathlib import Path

import nbformat

In [2]:
# import paths 
PROJECT_ROOT = Path.cwd().parent  # letterboxd-stats
sys.path.insert(0, str(PROJECT_ROOT))

from config.paths import RAW_DATA_DIR, PROCESSED_DATA_DIR

In [3]:
diary_df = pd.read_csv(RAW_DATA_DIR.joinpath('diary.csv'))

In [4]:
diary_df.head(2)

,Date,Name,Year,Letterboxd URI,Rating,Rewatch,Tags,Watched Date
0,2024-06-04,Jaws,1975,https://boxd.it/6BITLZ,3.0,NaN,NaN,2024-06-03
1,2024-07-15,Twister,1996,https://boxd.it/6SrRZT,2.5,Yes,NaN,2024-07-14


In [5]:
# make it easier for work with columns by renaming them to lowercase and replacing spaces with underscores
diary_df.columns = diary_df.columns.str.lower().str.replace(' ', '_')

# format columns correctly 
# date and watched_date to datetime
diary_df['date'] = pd.to_datetime(diary_df['date'], format='%Y-%m-%d')
diary_df['watched_date'] = pd.to_datetime(diary_df['watched_date'], format='%Y-%m-%d')
# rewatch to boolean
diary_df.loc[diary_df['rewatch'].isna(), 'rewatch'] = False
diary_df.loc[diary_df['rewatch']=='Yes', 'rewatch'] = True
diary_df['rewatch'] = diary_df['rewatch'].astype(bool)

# tags has a list of tags separated by commas
diary_df['tags'] = diary_df['tags'].str.replace(', ', ',')

diary_df['tags'] = diary_df['tags'].str.split(',')
# unique identifier for movies
# we'll say that a movie with the same name and the same year is the same movie since there's no other distinguishing feature
diary_df['unique_movies'] = diary_df['name'] + ' (' + diary_df['year'].astype(str) + ')'


# all time watches per movie

In [6]:
watches = diary_df.value_counts(['unique_movies']).reset_index(name='watch_count_per_movie').sort_values('watch_count_per_movie', ascending=False)

watch_count_per_movie = watches['watch_count_per_movie'].value_counts().sort_index(ascending=False).reset_index(name='watches')

fig = px.pie(watch_count_per_movie, 
             values='watches', 
             names=watch_count_per_movie.index, 
             title='Distribution of Watch Counts per Movie')

fig.update_layout(
    legend=dict(
        title=dict(text='Watches per Movie') 
    )
)

fig.update_layout(
    annotations=[
        dict(
            text='all time in diary',  
            x=-0.15,  
            y=1.175, 
            xref='paper',
            yref='paper',
            showarrow=False,
            font=dict(size=10)
        )
    ]
)

fig.show()

# top 10 movies

In [7]:
top10_movies = watches.head(10)

# if there's not 10 movies with multiple watches, show less than 10 movies 
top10_movies = top10_movies[top10_movies['watch_count_per_movie']>1]
top_x_movies = len(top10_movies)

fig = px.bar(top10_movies, 
             x='unique_movies', 
             y='watch_count_per_movie', 
             title=f'Top {top_x_movies} Most Watched Movies with more than 1 watch',
             labels={'name': 'Movie Name', 'watch_count_per_movie': 'Number of Watches'})


fig.show()

In [8]:
# ratings of top x movies 
top10_movies_list = list(top10_movies['unique_movies'])

top_x_movies_ratings = diary_df[diary_df['unique_movies'].isin(top10_movies_list)]
top_x_movies_ratings

fig = px.scatter(top_x_movies_ratings, 
             x='watched_date', 
             y='rating', 
             color='unique_movies',
             title=f'Ratings for Top {top_x_movies} Most Watched Movies with more than 1 watch',
             labels={'name': 'Movie Name', 'watch_count_per_movie': 'Number of Watches'})
fig.show()

In [9]:
# movies per day 
movies_per_day = diary_df['watched_date'].value_counts().sort_index()
fig = px.line(movies_per_day, 
              x=movies_per_day.index, 
              y=movies_per_day.values, 
              title='Movies Watched per Day',
              labels={'x': 'Watch Date', 'y': 'Number of Movies Watched'})
fig.show()

In [10]:
# top 10 dates with the most movies watched 
top10_dates = movies_per_day.sort_values(ascending=False).head(10).reset_index()
top10_dates.columns = ['watched_date', 'number_of_movies_watched']
top10_dates_fig = px.bar(top10_dates, 
                    x='watched_date', 
                    y='number_of_movies_watched',
                    title='Top 10 Dates with Most Movies Watched',
                    labels={'watched_date': 'Watch Date', 'number_of_movies_watched': 'Number of Movies Watched'})
top10_dates_fig.show()

In [11]:
# rewatch percentage 
rewatches = diary_df.value_counts('rewatch').reset_index(name='rewatch_count')
fig = px.pie(rewatches, 
    values='rewatch_count', 
    names='rewatch',
    title='Rewatch vs. First-time Watch Distribution')

fig.update_layout(
    legend=dict(
        title=dict(text='Rewatch') 
    )
)

fig.show()

In [12]:
used_tags = diary_df['tags'].explode().value_counts().reset_index(name='tag_count')
fig = px.bar(used_tags, 
       x='tags', 
       y='tag_count',
       title='Used tags count',
       labels={'tags': 'Tag', 'tag_count': 'Count'})

fig.show()

# Genres 

In [25]:
import requests
from bs4 import BeautifulSoup
def extract_base_url(letterboxd_url):
    user_name = letterboxd_url.split('/')[-4]
    letterboxd_url = letterboxd_url.replace('/'+user_name,'')
    return letterboxd_url

# ========== scrape letterboxd ==========
def get_letterboxd_film_info(url):
    url = extract_base_url(letterboxd_url)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html.parser")
    
    # Titel
    title_tag = soup.find("h1", {"class": "headline-1"})
    title = title_tag.text.strip() if title_tag else None
    
    # Jahr
    year_tag = soup.find("small", {"class": "number"})
    year = year_tag.text.strip() if year_tag else None
    
    return title, year

# Beispiel-URL
letterboxd_url = "https://letterboxd.com/evame/film/jaws/"
title, year = get_letterboxd_film_info(letterboxd_url)
print(f"Letterboxd: {title} ({year})")


# # ========== Schritt 2: OMDb API nutzen ==========
# def get_omdb_info(title, year, api_key):
#     url = f"http://www.omdbapi.com/?t={title}&y={year}&apikey={api_key}"
#     r = requests.get(url)
#     data = r.json()
    
#     if data.get("Response") == "True":
#         return data
#     else:
#         return None

# # OMDb API Key
# omdb_api_key = "DEIN_OMDB_API_KEY"
# omdb_data = get_omdb_info(title, year, omdb_api_key)

# if omdb_data:
#     print("OMDb Daten:")
#     print(f"Title: {omdb_data['Title']}")
#     print(f"Year: {omdb_data['Year']}")
#     print(f"IMDb ID: {omdb_data['imdbID']}")
#     print(f"Director: {omdb_data['Director']}")
#     print(f"Genre: {omdb_data['Genre']}")
#     print(f"Plot: {omdb_data['Plot']}")
# else:
#     print("Film nicht gefunden auf OMDb.")


IMDB gefunden!
1 Treffer gefunden:

Treffer 1:
...n 1976   Theatrical            124 mins   More at IMDb TMDB  Report this page  ...
1 Treffer gefunden:

Treffer 1:
... Emirates      09 Dec 2015   Premiere Dubai International Film Festival           Uruguay      01 Jun 1976   Theatrical            124 mins   More at IMDb TMDB  Report this page  ...
IMDB nicht gefunden.
1 Treffer gefunden:

Treffer 1:
... Emirates      09 Dec 2015   Premiere Dubai International Film Festival           Uruguay      01 Jun 1976   Theatrical            124 mins   More at IMDb TMDB  Report this page  ...
IMDB nicht gefunden.
Gefundene Links: 230
/
/user/request-password-reset

#
/list/new/
#
#
/settings/


//www.youtube.com/embed/N9hzLQUGr34?rel=0&wmode=transparent
https://www.justwatch.com
/films/year/1975/
/director/steven-spielberg/
/film/jaws/
/film/jaws/crew/
/film/jaws/details/
/film/jaws/genres/
/film/jaws/releases/
/actor/roy-scheider/
/actor/robert-shaw/
/actor/richard-dreyfuss/
/actor/lorraine-gar

In [26]:
# get all href - shows the imdb href too! also other very interesting links, e.g. showing all actors and the crew,..
hrefs = [a["href"] for a in soup.find_all("a", href=True)]

print(f"Found Links: {len(hrefs)}")
for link in hrefs:
    print(link)



NameError: name 'soup' is not defined